# COLAB/PARTICIPANTE

Para cada selfie não processada, baixa a selfie e procura fotos desta pessoa no evento. Ao encontrar as imagens, envia elas para o e-mail.

# Setup

In [ ]:
!pip install --quiet google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
!pip install --quiet --upgrade google-api-python-client
!pip install --quiet python-docx deepface unidecode

from google.colab import auth
import googleapiclient
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from googleapiclient.http import MediaIoBaseDownload

import io
import os
import zipfile
import unidecode
import numpy as np
import pandas as pd
from tqdm import tqdm
from docx import Document
from deepface import DeepFace
import matplotlib.pyplot as plt

# for email stuff
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText
import getpass

In [ ]:
# Necessary paths with respective Google Drive IDs (follows this: drive.google.com/drive/u/1/folders/{ID})
SHOWFACE_FOLDER = '1iUXrRc9p0CL-xSXXBuPjjQ0xHtL9YimY' # main showface folder
SELFIES_FOLDER = '1BufjlvWUhOukoZlr15aa2O_3RswchW3AxBhFOn0VNqRwVUVR3Wv4x95RzQk7W0qrrZ64VDZB'
SELFIES_DOCS = '1HCVTIHQ0DIQRWyl1vAuW-XU6anK_CqD_GIGGCnkcV6E'
ALBUMS_FOLDER = '1ra03dT_nthAPFHh_zFeKfgT91_kiU2Cq'

### Useful functions

In [ ]:
def list_files_in_shared_folder(folder_id):
    query = f"'{folder_id}' in parents and trashed = false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])
    if not files:
        print("No files found in the shared folder.")
    else:
        print("Files in the shared folder:")
        for file in files:
            print(f"Name: {file['name']}, ID: {file['id']}")
    return files

In [ ]:
def download_file(file_id, file_name, extension):
    """Downloads a file from Google Drive by ID."""
    request = drive_service.files().get_media(fileId=file_id)
    file_path = f"/content/{file_name}.{extension}"

    with open(file_path, 'wb') as file:
        downloader = googleapiclient.http.MediaIoBaseDownload(file, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}% complete.")

    print(f"File downloaded to {file_path}")
    return file_path

In [ ]:
def get_google_sheet_as_dataframe(file_id):
    """Download a Google Spreadsheet as CSV and return it as a pandas DataFrame."""
    try:
        # Export the spreadsheet as CSV
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='text/csv'
        )
        # Save the exported CSV to memory
        csv_data = request.execute()

        # Load the CSV data into a pandas DataFrame
        from io import StringIO
        table_df = pd.read_csv(StringIO(csv_data.decode('utf-8')))

        return table_df

    except Exception as e:
        print(f"An error occurred while processing the spreadsheet with ID {file_id}: {e}")
        return None


In [ ]:
def download_folder_contents(folder_id, download_path):
    """Downloads all files from a specified Google Drive folder."""
    # Ensure the download path exists
    os.makedirs(download_path, exist_ok=True)

    # Query to list all files in the specified folder
    query = f"'{folder_id}' in parents and trashed = false"
    results = drive_service.files().list(pageSize=1000, q=query, fields="files(id, name)").execute()
    files = results.get('files', [])

    if not files:
        print("No files found in the specified folder.")
        return

    print(f"Found {len(files)} files in the folder. Starting download...")

    for file in files:
        file_id = file['id']
        file_name = file['name']

        # Request to download the file
        request = drive_service.files().get_media(fileId=file_id)
        file_path = os.path.join(download_path, file_name)

        with io.FileIO(file_path, 'wb') as file_obj:
            downloader = MediaIoBaseDownload(file_obj, request)
            done = False
            while not done:
                _, done = downloader.next_chunk()

    print("All files downloaded successfully.")


In [ ]:
def send_email_with_images(recipient_email, image_paths, sender_email, sender_password, person_name, event_name):
    """Sends an email with images as attachments"""

    # SMTP server settings
    smtp_server = "smtp.gmail.com"
    smtp_port = 587  # Port for TLS

    # Creating the email body
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = "Sua solicitação de imagens no Showface"  # Email subject

    email_body = (
        f"Olá {person_name}, tudo bem?\n\n"
        f"Vimos que você solicitou suas fotos para o evento {event_name}. Você pode encontrá-las em anexo, no e-mail! \n\n"
        "Showface"
    )
    msg.attach(MIMEText(email_body, 'plain'))

    # Attach images to the email
    valid_attachments = True

    for image_path in image_paths:
        if not os.path.isfile(image_path):
            print(f"Error: File not found or invalid path: {image_path}")
            valid_attachments = False
            continue

        try:
            with open(image_path, "rb") as file:
                attachment = MIMEBase('application', 'octet-stream')
                attachment.set_payload(file.read())
                encoders.encode_base64(attachment)
                attachment_name = os.path.basename(image_path)
                attachment.add_header(
                    'Content-Disposition', f'attachment; filename={attachment_name}'
                )
                msg.attach(attachment)
        except Exception as e:
            print(f"Error attaching file {image_path}: {e}")
            valid_attachments = False

    # Send the email only if all attachments are valid
    if valid_attachments:
        try:
            with smtplib.SMTP(smtp_server, smtp_port) as server:
                server.starttls()  # Secure the connection
                server.login(sender_email, sender_password)
                server.sendmail(sender_email, recipient_email, msg.as_string())
                print(f"Email successfully sent to {recipient_email}.")
        except Exception as e:
            print(f"Error sending email: {str(e)}")
    else:
        print("Email sending canceled due to issues with attachments.")

In [ ]:
def update_google_sheet_from_dataframe(spreadsheet_id, dataframe):
    """Update a Google Spreadsheet with the contents of a pandas DataFrame, starting at A1."""
    try:

        dataframe = dataframe.fillna("")

        # Convert the DataFrame to a list of lists (Google Sheets format)
        values = [dataframe.columns.tolist()] + dataframe.values.tolist()

        # Create the request body
        body = {
            "majorDimension": "ROWS",
            "values": values
        }

        # Use the Sheets API to update the entire spreadsheet, starting from A1
        sheets_service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range="A1",  # Always starts at the first cell
            valueInputOption="RAW",
            body=body
        ).execute()

        print(f"Spreadsheet {spreadsheet_id} updated successfully.")

    except Exception as e:
        print(f"An error occurred while updating the spreadsheet with ID {spreadsheet_id}: {e}")


## Authentication

In [ ]:
# Authenticate with Colab's native method
auth.authenticate_user()

# Initialize the Drive API
SCOPES = ['https://www.googleapis.com/auth/drive']
drive_service = build('drive', 'v3')
sheets_service = build('sheets', 'v4')
print("Authentication complete!")

Authentication complete!


In [ ]:
# If everything is correct, should show the shared folder items
list_files_in_shared_folder(SHOWFACE_FOLDER)

Files in the shared folder:
Name: Envio de Selfie - Usuário (respostas), ID: 1HCVTIHQ0DIQRWyl1vAuW-XU6anK_CqD_GIGGCnkcV6E
Name: Envio de Álbum - Organizador (respostas), ID: 1RcmynInVP3jFfeqd7g8ocq8El1iOg2Qa3VpBGc-_JNY
Name: Envio de Selfie - Usuário, ID: 1BNUjfznWTN6AcPNi2nABAcRUJ08CnNaoacZ-PH6kqzM
Name: Albums, ID: 1ra03dT_nthAPFHh_zFeKfgT91_kiU2Cq
Name: Colabs, ID: 1HXfpPcdXlm8LUGjPFux0vdaPyrVty2cf
Name: Envio de Álbum - Organizador, ID: 18sxdkn-rJBIpV9zsM7VYmrNdp33M3RnCgFfDiImG4ZQ
Name: Envio de Selfie - Usuário (File responses), ID: 1TIg8DPiAAzOtyOnRDDkW26F3z0by6iA1X_HskpxuPPMt1jyj76mrrB-2F6sLGSHGr9vTQntU
Name: Envio de Álbum (File responses), ID: 1pX9LdtpL9D30jHKcfBX5me8wspIDwwUZmVi2fM0NDxYpVgaG2lRpXLS3rJuLay3jDW2V2mQ8
Name: Oscar_Emmy_2023_2024, ID: 1gehG-x3DdolRQKfMzeTJh3zRovBSNltH


[{'id': '1HCVTIHQ0DIQRWyl1vAuW-XU6anK_CqD_GIGGCnkcV6E',
  'name': 'Envio de Selfie - Usuário (respostas)'},
 {'id': '1RcmynInVP3jFfeqd7g8ocq8El1iOg2Qa3VpBGc-_JNY',
  'name': 'Envio de Álbum - Organizador (respostas)'},
 {'id': '1BNUjfznWTN6AcPNi2nABAcRUJ08CnNaoacZ-PH6kqzM',
  'name': 'Envio de Selfie - Usuário'},
 {'id': '1ra03dT_nthAPFHh_zFeKfgT91_kiU2Cq', 'name': 'Albums'},
 {'id': '1HXfpPcdXlm8LUGjPFux0vdaPyrVty2cf', 'name': 'Colabs'},
 {'id': '18sxdkn-rJBIpV9zsM7VYmrNdp33M3RnCgFfDiImG4ZQ',
  'name': 'Envio de Álbum - Organizador'},
 {'id': '1TIg8DPiAAzOtyOnRDDkW26F3z0by6iA1X_HskpxuPPMt1jyj76mrrB-2F6sLGSHGr9vTQntU',
  'name': 'Envio de Selfie - Usuário (File responses)'},
 {'id': '1pX9LdtpL9D30jHKcfBX5me8wspIDwwUZmVi2fM0NDxYpVgaG2lRpXLS3rJuLay3jDW2V2mQ8',
  'name': 'Envio de Álbum (File responses)'},
 {'id': '1gehG-x3DdolRQKfMzeTJh3zRovBSNltH', 'name': 'Oscar_Emmy_2023_2024'}]

# Pre-process
Faz download das selfies ainda não processadas e seus respectivos álbuns de acordo com o docs de respostas.

In [ ]:
answers = get_google_sheet_as_dataframe(SELFIES_DOCS)
filtered_answers = answers[(answers["PROCESSADO?"] == False) & (answers["Qual é o seu nome?"].notna())] # gets all rows that are "PROCESSADO?" = False and Name is not NaN
filtered_answers.head()

,Carimbo de data/hora,Qual é o seu nome?,Informe o seu email:,Envie uma selfie do seu rosto:,Qual o nome do evento?,PROCESSADO?
10,18/12/2024 23:54:46,Florence Pugh,flfp@cin.ufpe.br,https://drive.google.com/open?id=1pxr-FyUKUInS...,Oscar 2024,False
11,18/12/2024 23:55:17,Zendaya,flfp@cin.ufpe.br,https://drive.google.com/open?id=15IkCFFqdT-3h...,Oscar 2024,False
12,18/12/2024 23:55:41,Jeremy,flfp@cin.ufpe.br,https://drive.google.com/open?id=1A1KcHwjLs6kp...,Grammy,False


In [ ]:
answers_to_process = []
albums = list_files_in_shared_folder(ALBUMS_FOLDER)
print(albums)

for index, answer in filtered_answers.iterrows():
  file_id = answer["Envie uma selfie do seu rosto:"].split("=")[1]
  # download reference iamge
  file_path = download_file(file_id, unidecode.unidecode(answer["Qual é o seu nome?"]), 'png')

  # download album
  album = answer['Qual o nome do evento?']
  album_id = ''
  for album_file in albums:
    print(album, album_file['name'])
    if album_file['name'] == album:
      album_id = album_file['id']
      break

  album_subfolders = list_files_in_shared_folder(album_id)
  cropped_subfolder_id = ''
  original_subfolder_id = ''
  for subfolder in album_subfolders:
    if subfolder['name'] == 'cropped':
      cropped_subfolder_id = subfolder['id']
    elif subfolder['name'] == 'images':
      original_subfolder_id = subfolder['id']

  if not os.path.isdir(f'/content/albums/{album}'):
    download_folder_contents(original_subfolder_id, f'/content/albums/{album}')

  answers_to_process.append({
      'name': answer['Qual é o seu nome?'],
      'email': answer['Informe o seu email:'],
      'selfie_path': file_path,
      'album_name': album,
      'album_path': f'/content/albums/{album}',
      'album_original_driveid': original_subfolder_id
  })

print(answers_to_process)

Files in the shared folder:
Name: Oscar 2024, ID: 1lWirVxk7NTZSSHfHjkQ4rayT8UzOuZfc
Name: Grammy, ID: 15VJ2HN6oKgxr8cVYCgsjRsARDQbeSu1K
[{'id': '1lWirVxk7NTZSSHfHjkQ4rayT8UzOuZfc', 'name': 'Oscar 2024'}, {'id': '15VJ2HN6oKgxr8cVYCgsjRsARDQbeSu1K', 'name': 'Grammy'}]
Download 100% complete.
File downloaded to /content/Florence Pugh.png
Oscar 2024 Oscar 2024
Files in the shared folder:
Name: images, ID: 1EwfKs3O6f08QRttvLFGKHy-jop0kckQB
Download 100% complete.
File downloaded to /content/Zendaya.png
Oscar 2024 Oscar 2024
Files in the shared folder:
Name: images, ID: 1EwfKs3O6f08QRttvLFGKHy-jop0kckQB
Download 100% complete.
File downloaded to /content/Jeremy.png
Grammy Oscar 2024
Grammy Grammy
Files in the shared folder:
Name: images, ID: 1B4Q194ASxcxL2T0RRKBsvzjSvUcoT29O
[{'name': 'Florence Pugh', 'email': 'flfp@cin.ufpe.br', 'selfie_path': '/content/Florence Pugh.png', 'album_name': 'Oscar 2024', 'album_path': '/content/albums/Oscar 2024', 'album_original_driveid': '1EwfKs3O6f08QRttvLFG

# Face verification e envio por e-mail

In [ ]:
# Solicitar informações para envio do e-mail
sender_email = input("Digite seu e-mail: ")
sender_password = getpass.getpass("Digite sua senha (será ocultada): ")

# COMPARAR FOTO(PATH) DE 1 ROSTO COM LISTA DE FOTOS(PATHS) DE 1 ROSTO E RETORNA LISTA DE FOTOS CORRESPONDENTES(PATHS)
for answer in answers_to_process:
  client_path = answer['selfie_path']  # imagem referência

  matches = DeepFace.find(
    img_path = client_path,
    db_path = answer['album_path'],
    enforce_detection=False,
    threshold=0.5,
    model_name = 'Facenet',
  )[0]

  images_paths = list(matches['identity'])

  print("Found: ", images_paths)

  # https://www.google.com/settings/security/lesssecureapps
  send_email_with_images(answer['email'], images_paths, sender_email, sender_password, answer['name'], answer['album_name'])


Digite seu e-mail: flfp@cin.ufpe.br
Digite sua senha (será ocultada): ··········
24-12-19 03:36:34 - Searching /content/Florence Pugh.png in 165 length datastore
24-12-19 03:36:35 - find function duration 1.1298203468322754 seconds
Found:  ['/content/albums/Oscar 2024/MV5BZDJjZDY0YWItYmIzMC00ZGJiLWE4ZTYtMWVjMWMxMmNmYmVkXkEyXkFqcGc@._V1_.jpg', '/content/albums/Oscar 2024/MV5BZGMxYzJjNjUtNTgyMy00YzQzLWE1MzAtNzJiNzVlNTg5NjA1XkEyXkFqcGc@._V1_.jpg', '/content/albums/Oscar 2024/MV5BZmMyNDc4YWQtMTQyZS00MTg4LTllNGYtMjU2MzhmY2NkMzU2XkEyXkFqcGc@._V1_QL75_UX692_.jpg']
Email successfully sent to flfp@cin.ufpe.br.
24-12-19 03:36:39 - Searching /content/Zendaya.png in 165 length datastore
24-12-19 03:36:40 - find function duration 0.9786741733551025 seconds
Found:  ['/content/albums/Oscar 2024/MV5BNDY2YWE4NTctNzU0Yi00MzE4LTljMDEtN2M0MWVmMTg4NGI1XkEyXkFqcGc@._V1_.jpg', '/content/albums/Oscar 2024/MV5BZjkyYjIwZjktYjhiNy00MGZiLThmNmEtMmFmNzU5ZjNhMTA4XkEyXkFqcGc@._V1_.jpg']
Email successfully sent to fl

# Update answers docs
Mark images as processed

In [ ]:
for idx, answer in answers.iterrows():
  if not isinstance(answer['Qual é o seu nome?'], float): # checks for NaNs
    answers.at[idx, 'PROCESSADO?'] = True

update_google_sheet_from_dataframe(SELFIES_DOCS, answers)

Spreadsheet 1HCVTIHQ0DIQRWyl1vAuW-XU6anK_CqD_GIGGCnkcV6E updated successfully.
